# Derive equation of motion of monopod

In [1]:
# import libraries
import sympy as sym
import numpy as np

sym.init_printing()
from IPython.display import display #for pretty printing

# create symbolic variables

# system parameters
g = sym.symbols('g')
mb,mlt,mlb = sym.symbols(['m_{body}','m_{leg1}','m_{leg2}']) # mass
lb,llt,llb = sym.symbols(['l_{body}','l_{leg1}','l_{leg2}']) # length
Inb,Inlt,Inlb = sym.symbols(['I_{body}','I_{leg1}','I_{leg2}']) # moment of intertia

# generalized coordinates
x,y,thb,thlt,thlb = sym.symbols(['x','y','\\theta_{body}','\\theta_{legtop}','\\theta_{legbot}']) 
dx,dy,dthb,dthlt,dthlb = sym.symbols(['\dot{x}','\dot{y}','\dot{\\theta}_{body}','\dot{\\theta}_{legtop}','\dot{\\theta}_{legbot}']) 
ddx,ddy,ddthb,ddthlt,ddthlb = sym.symbols(['\ddot{x}','\ddot{y}','\ddot{\\theta}_{body}','\ddot{\\theta}_{legtop}','\ddot{\\theta}_{legbot}']) 

q = sym.Matrix([[x],[y],[thb],[thlt],[thlb]])
dq = sym.Matrix([[dx],[dy],[dthb],[dthlt],[dthlb]])
ddq = sym.Matrix([[ddx],[ddy],[ddthb],[ddthlt],[ddthlb]])

# forces
# total joint action = actuator + rebound, but that will be dealt with elsewhere
tauh,tauk,GRFx,GRFy = sym.symbols(['\\tau_{hip}','\\tau_{knee}','G_x','G_y']) 

# STEP 1: position vectors ri = [x,y,theta] (world frame)
rb = sym.Matrix([[x],
                [y],
                [thb]])

rl1 = sym.Matrix([[x + 0.5*llt*sym.sin(thb + thlt)],
                [y - 0.5*llt*sym.cos(thb + thlt)],
                [thb + thlt]])

rl2 = sym.Matrix([[x + llt*sym.sin(thb + thlt)+0.5*llb*sym.sin(thb + thlt + thlb)],
                [y - llt*sym.cos(thb + thlt) - 0.5*llb*sym.cos(thb + thlt + thlb)],
                [thb + thlt + thlb]])

# the Jacobians
Jb = rb.jacobian(q)
Jl1 = rl1.jacobian(q)
Jl2 = rl2.jacobian(q)

# STEP 2: generate expressions for the system space velocities from the jacobians
vb = Jb*dq
vl1 = Jl1*dq
vl2 = Jl2*dq

# STEP 3: generate expressions for the kinetic and potential energy
# mass vectors
Mb = sym.Matrix([[mb,mb,Inb]])
Ml1 = sym.Matrix([[mlt,mlt,Inlt]])
Ml2 = sym.Matrix([[mlb,mlb,Inlb]])

T = 0.5*Mb*sym.matrix_multiply_elementwise(vb,vb) + 0.5*Ml1*sym.matrix_multiply_elementwise(vl1,vl1) + 0.5*Ml2*sym.matrix_multiply_elementwise(vl2,vl2)
T = T[0]
V = mb*g*rb[1] + mlt*g*rl1[1] + mlb*g*rl2[1]


# STEP 4: calculate each term of the Lagrange equation
# term 1
Lg1 = sym.zeros(len(q),1)
for i in range(len(q)):
    dT_ddq = sym.Matrix([sym.diff(T,dq[i])]) # get partial of T in dq_i
    Lg1[i] = dT_ddq.jacobian(q)*dq + dT_ddq.jacobian(dq)*ddq #...then get time derivative of that partial

# term 3
Lg3 = sym.Matrix([T]).jacobian(q).transpose() # partial of T in q

# term 4
Lg4 = sym.Matrix([V]).jacobian(q).transpose() # partial of U in q

# STEP 5: generalized forces

# force vectors for each link
tau_b = sym.Matrix([[0],[0],[-tauh]])
tau_l1 = sym.Matrix([[0],[0],[tauh-tauk]])
tau_l2 = sym.Matrix([[0],[0],[tauk]])

GRF_l2 = sym.Matrix([[GRFx],[GRFy],[0.5*llb*GRFx*sym.cos(thb+thlt+thlb)+0.5*llb*GRFy*sym.sin(thb+thlt+thlb)]])

Q = sym.zeros(len(q),1)
for j in range(len(q)):
    Q[j] = tau_b.transpose()*Jb[:,j]+(tau_l1).transpose()*Jl1[:,j]+(tau_l2+GRF_l2).transpose()*Jl2[:,j]

# AND combine!
EOM = Lg1 - Lg3 + Lg4 - Q

EOMs = sym.zeros(len(q),1)
for j in range(len(q)):
    EOMs[j] = EOM[j].simplify()

In [2]:
# Position and velocity of monopod foot

pfoot = sym.Matrix([[x + llt*sym.sin(thb + thlt) + llb*sym.sin(thb + thlt + thlb)],
                    [y - llt*sym.cos(thb + thlt) - llb*sym.cos(thb + thlt + thlb)]])

vfoot = pfoot.jacobian(q)*dq

pfoot
# vfoot

⎡l_{leg1}⋅sin(\theta_{body} + \theta_{legtop}) + l_{leg2}⋅sin(\theta_{body} + 
⎢                                                                             
⎣-l_{leg1}⋅cos(\theta_{body} + \theta_{legtop}) - l_{leg2}⋅cos(\theta_{body} +

\theta_{legbot} + \theta_{legtop}) + x ⎤
                                       ⎥
 \theta_{legbot} + \theta_{legtop}) + y⎦

In [3]:
import pickle as pkl

data = {"EOMs":EOMs, "pfoot":pfoot, "vfoot":vfoot}

outfile = open('monopod_EOMs','wb')
pkl.dump(data,outfile)
outfile.close()

In [4]:
infile = open('monopod_EOMs','rb')
data = pkl.load(infile)
infile.close()

EOMs = data['EOMs']
pfoot = data['pfoot']
vfoot = data['vfoot']


In [5]:
var = [mb, mlt, mlb, lb, llt, llb, Inb, Inlt, Inlb, x, y, thb, thlt, thlb, dx, dy, dthb, dthlt, dthlb]
var_values = [(g, 9.81),(mb,1),(mlt,1),(mlb,1),(lb,1),(llt,1),(llb,1),(Inb,1),(Inlt,1),(Inlb,1)]

In [9]:
EOM_sub = EOM.subs(var_values)
EOM_sub

⎡                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢      -Gₓ⋅(cos(\theta_{body} + \theta_{legtop}) + 0.5⋅cos(\theta_{body} + \th
⎢                                                                             
⎢                                                                             
⎢-Gₓ⋅(cos(\theta_{body} + \theta_{legtop}) + 0.5⋅cos(\theta_{body} + \theta_{l
⎢                                                                             
⎢                                                                             
⎣                                                                             

                                                   

In [60]:
EOM_sub.evalf()

⎡                             -1.11921650312094⋅\ddot{\theta}_{body} - 0.49499
⎢                                                                             
⎢                         1.43450614426846⋅\ddot{\theta}_{body} + 0.0705600040
⎢                                                                             
⎢5.04030230586814⋅\ddot{\theta}_{body} + 1.52015115293407⋅\ddot{\theta}_{legbo
⎢                                                                             
⎢4.04030230586814⋅\ddot{\theta}_{body} + 1.52015115293407⋅\ddot{\theta}_{legbo
⎢                                                                             
⎣   1.52015115293407⋅\ddot{\theta}_{body} + 1.25⋅\ddot{\theta}_{legbot} + 1.52

6248300223⋅\ddot{\theta}_{legbot} - 1.11921650312094⋅\ddot{\theta}_{legtop} + 
                                                                              
299336⋅\ddot{\theta}_{legbot} + 1.43450614426846⋅\ddot{\theta}_{legtop} + 3.0⋅
                                                   

In [ ]:
EOM_solve = sym.solve(EOM_sub, ddq)

In [ ]:
func_map = {'sin':sin, 'cos':cos} 

sym_list = [g,mb,mlt,mlb,lb,llt,llb,Inb,Inlt,Inlb,
            x,y,thb,thlt,thlb,
            dx,dy,dthb,dthlt,dthlb,
            ddx,ddy,ddthb,ddthlt,ddthlb,
            tauh,tauk,GRFx,GRFy]
            
lambEOM_xb = sym.lambdify(sym_list,EOMs[0],modules = [func_map])
lambEOM_yb = sym.lambdify(sym_list,EOMs[1],modules = [func_map])
lambEOM_thb = sym.lambdify(sym_list,EOMs[2],modules = [func_map])
lambEOM_thlt = sym.lambdify(sym_list,EOMs[3],modules = [func_map])
lambEOM_thlb = sym.lambdify(sym_list,EOMs[4],modules = [func_map])


In [ ]:
def update(q, dq):
    for i in range(num_vars):
        ddq[i] = EOM_lambdified[i](q, dq)